<h1>Detecting Animals</h1>

<h2>Imporing Libaries</h2>

In [1]:
import matplotlib
matplotlib.use("Agg")
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2

<h2>Getting the data</h2>

In [2]:
data =[]
labels = []

image_paths = sorted(list(paths.list_images("./animals")))
random.seed(42)
random.shuffle(image_paths)

for imgpath in image_paths:
    #Loading the image, resizing it to a fixed size and then flattening it
    image = cv2.imread(imgpath)
    image = cv2.resize(image, (32, 32)).flatten()
    data.append(image)
    #Getting the labels and appending them to the labels list
    label = imgpath.split(os.path.sep)[-2]
    labels.append(label)


In [3]:
data = np.array(data, dtype='float')/255.0
labels = np.array(labels)


<h2>Spliting the data into train and test sets</h2>

In [4]:
(train_X, test_X, train_Y, test_Y) = train_test_split(data, labels, test_size=0.25, random_state=42)

<h2>Performing one-hot encodings</h2>

In [5]:
lb = LabelBinarizer()
train_Y = lb.fit_transform(train_Y)
test_Y = lb.transform(test_Y)

<h2>Defining the model</h2> 

In [7]:
model = Sequential()
model.add(Dense(1024, input_shape=(3072,) , activation="sigmoid"))
model.add(Dense(512, activation = 'sigmoid'))
model.add(Dense(len(lb.classes_), activation = 'softmax'))

<h2>Compiling</h2>

In [14]:
opt = SGD(learning_rate = 0.01)
model.compile(loss="categorical_crossentropy", optimizer = opt, metrics=["accuracy"])

<h2>Fitting the data</h2>

In [15]:
H = model.fit(x= train_X, y= train_Y, validation_data=(test_X, test_Y), epochs = 100, batch_size = 32)

Epoch 1/100
71/71 [==============================] - 2s 15ms/step - loss: 1.1034 - accuracy: 0.3316 - val_loss: 1.1243 - val_accuracy: 0.3147
Epoch 2/100
71/71 [==============================] - 1s 10ms/step - loss: 1.0850 - accuracy: 0.3804 - val_loss: 1.1222 - val_accuracy: 0.3147
Epoch 3/100
71/71 [==============================] - 1s 7ms/step - loss: 1.0776 - accuracy: 0.3929 - val_loss: 1.0552 - val_accuracy: 0.3853
Epoch 4/100
71/71 [==============================] - 1s 8ms/step - loss: 1.0566 - accuracy: 0.4484 - val_loss: 1.0662 - val_accuracy: 0.3733
Epoch 5/100
71/71 [==============================] - 1s 8ms/step - loss: 1.0431 - accuracy: 0.4502 - val_loss: 1.0985 - val_accuracy: 0.3147
Epoch 6/100
71/71 [==============================] - 1s 7ms/step - loss: 1.0311 - accuracy: 0.4813 - val_loss: 1.0854 - val_accuracy: 0.3253
Epoch 7/100
71/71 [==============================] - 1s 8ms/step - loss: 1.0182 - accuracy: 0.4831 - val_loss: 0.9939 - val_accuracy: 0.5453
Epoch 8/100

<h2>Evaluating the model</h2>

In [23]:
predictions = model.predict(x=test_X, batch_size = 32)
print(classification_report(test_Y.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

#plotting the training loss and accuracy
N = np.arange(0, 100)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("plot.png")


              precision    recall  f1-score   support

        cats       0.56      0.59      0.57       236
        dogs       0.51      0.37      0.43       236
       panda       0.70      0.84      0.77       278

    accuracy                           0.61       750
   macro avg       0.59      0.60      0.59       750
weighted avg       0.60      0.61      0.60       750



In [22]:
model.save("plot.png", save_format="h5")
f = open("model.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()